# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 21 2022 11:00AM,246930,10,0085995868,ISDIN Corporation,Released
1,Sep 21 2022 11:00AM,246930,10,0085995871,ISDIN Corporation,Released
2,Sep 21 2022 11:00AM,246930,10,0085995872,ISDIN Corporation,Released
3,Sep 21 2022 11:00AM,246930,10,0085995928,ISDIN Corporation,Released
4,Sep 21 2022 11:00AM,246930,10,0085995947,ISDIN Corporation,Released
5,Sep 21 2022 11:00AM,246930,10,0085996240,ISDIN Corporation,Released
6,Sep 21 2022 11:00AM,246930,10,0085996242,ISDIN Corporation,Released
7,Sep 21 2022 11:00AM,246930,10,0085996244,ISDIN Corporation,Released
8,Sep 21 2022 11:00AM,246930,10,0085999435,ISDIN Corporation,Released
9,Sep 21 2022 11:00AM,246930,10,0085999437,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,246926,Released,1
40,246927,Released,1
41,246928,Released,2
42,246929,Released,1
43,246930,Released,34


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246926,NaN,NaN,1.0
246927,NaN,NaN,1.0
246928,NaN,NaN,2.0
246929,NaN,NaN,1.0
246930,NaN,NaN,34.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246717,0.0,0.0,1.0
246727,0.0,1.0,0.0
246811,41.0,0.0,8.0
246828,15.0,57.0,5.0
246844,8.0,7.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246717,0,0,1
246727,0,1,0
246811,41,0,8
246828,15,57,5
246844,8,7,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246717,0,0,1
1,246727,0,1,0
2,246811,41,0,8
3,246828,15,57,5
4,246844,8,7,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246717,,,1
1,246727,,1,
2,246811,41,,8
3,246828,15,57,5
4,246844,8,7,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 21 2022 11:00AM,246930,10,ISDIN Corporation
34,Sep 21 2022 10:55AM,246929,10,ISDIN Corporation
35,Sep 21 2022 10:48AM,246928,10,ISDIN Corporation
37,Sep 21 2022 10:39AM,246927,10,"Citieffe, Inc."
38,Sep 21 2022 10:35AM,246926,10,Beach Products Inc
39,Sep 21 2022 10:32AM,246925,10,Bio-PRF
43,Sep 21 2022 10:17AM,246923,19,"Methapharm, Inc."
50,Sep 21 2022 10:11AM,246920,10,Bio-PRF
61,Sep 21 2022 9:57AM,246919,16,"SHL Pharma, LLC"
62,Sep 21 2022 9:39AM,246916,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 21 2022 11:00AM,246930,10,ISDIN Corporation,,,34
1,Sep 21 2022 10:55AM,246929,10,ISDIN Corporation,,,1
2,Sep 21 2022 10:48AM,246928,10,ISDIN Corporation,,,2
3,Sep 21 2022 10:39AM,246927,10,"Citieffe, Inc.",,,1
4,Sep 21 2022 10:35AM,246926,10,Beach Products Inc,,,1
5,Sep 21 2022 10:32AM,246925,10,Bio-PRF,,,4
6,Sep 21 2022 10:17AM,246923,19,"Methapharm, Inc.",,,7
7,Sep 21 2022 10:11AM,246920,10,Bio-PRF,,,11
8,Sep 21 2022 9:57AM,246919,16,"SHL Pharma, LLC",,1,
9,Sep 21 2022 9:39AM,246916,15,"Alliance Pharma, Inc.",,,31


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 11:00AM,246930,10,ISDIN Corporation,34,,
1,Sep 21 2022 10:55AM,246929,10,ISDIN Corporation,1,,
2,Sep 21 2022 10:48AM,246928,10,ISDIN Corporation,2,,
3,Sep 21 2022 10:39AM,246927,10,"Citieffe, Inc.",1,,
4,Sep 21 2022 10:35AM,246926,10,Beach Products Inc,1,,
5,Sep 21 2022 10:32AM,246925,10,Bio-PRF,4,,
6,Sep 21 2022 10:17AM,246923,19,"Methapharm, Inc.",7,,
7,Sep 21 2022 10:11AM,246920,10,Bio-PRF,11,,
8,Sep 21 2022 9:57AM,246919,16,"SHL Pharma, LLC",,1,
9,Sep 21 2022 9:39AM,246916,15,"Alliance Pharma, Inc.",31,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 11:00AM,246930,10,ISDIN Corporation,34,,
1,Sep 21 2022 10:55AM,246929,10,ISDIN Corporation,1,,
2,Sep 21 2022 10:48AM,246928,10,ISDIN Corporation,2,,
3,Sep 21 2022 10:39AM,246927,10,"Citieffe, Inc.",1,,
4,Sep 21 2022 10:35AM,246926,10,Beach Products Inc,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 11:00AM,246930,10,ISDIN Corporation,34.0,NaN,NaN
1,Sep 21 2022 10:55AM,246929,10,ISDIN Corporation,1.0,NaN,NaN
2,Sep 21 2022 10:48AM,246928,10,ISDIN Corporation,2.0,NaN,NaN
3,Sep 21 2022 10:39AM,246927,10,"Citieffe, Inc.",1.0,NaN,NaN
4,Sep 21 2022 10:35AM,246926,10,Beach Products Inc,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 11:00AM,246930,10,ISDIN Corporation,34.0,0.0,0.0
1,Sep 21 2022 10:55AM,246929,10,ISDIN Corporation,1.0,0.0,0.0
2,Sep 21 2022 10:48AM,246928,10,ISDIN Corporation,2.0,0.0,0.0
3,Sep 21 2022 10:39AM,246927,10,"Citieffe, Inc.",1.0,0.0,0.0
4,Sep 21 2022 10:35AM,246926,10,Beach Products Inc,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4197299,109.0,3.0,11.0
12,246872,1.0,0.0,0.0
15,493727,39.0,0.0,41.0
16,1234444,6.0,13.0,9.0
19,246923,7.0,0.0,0.0
20,740642,22.0,57.0,15.0
21,740322,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4197299,109.0,3.0,11.0
1,12,246872,1.0,0.0,0.0
2,15,493727,39.0,0.0,41.0
3,16,1234444,6.0,13.0,9.0
4,19,246923,7.0,0.0,0.0
5,20,740642,22.0,57.0,15.0
6,21,740322,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,109.0,3.0,11.0
1,12,1.0,0.0,0.0
2,15,39.0,0.0,41.0
3,16,6.0,13.0,9.0
4,19,7.0,0.0,0.0
5,20,22.0,57.0,15.0
6,21,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,109.0
1,12,Released,1.0
2,15,Released,39.0
3,16,Released,6.0
4,19,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,11.0,0.0,41.0,9.0,0.0,15.0,0.0
Executing,3.0,0.0,0.0,13.0,0.0,57.0,1.0
Released,109.0,1.0,39.0,6.0,7.0,22.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,11.0,0.0,41.0,9.0,0.0,15.0,0.0
1,Executing,3.0,0.0,0.0,13.0,0.0,57.0,1.0
2,Released,109.0,1.0,39.0,6.0,7.0,22.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,11.0,0.0,41.0,9.0,0.0,15.0,0.0
1,Executing,3.0,0.0,0.0,13.0,0.0,57.0,1.0
2,Released,109.0,1.0,39.0,6.0,7.0,22.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()